# Unstructured and Structured RAG with Intelligent Query Routing

This notebook demonstrates an advanced RAG pattern that intelligently routes queries between **structured** and **unstructured** knowledge bases using **Strands Agents** 

## Overview

The system uses custom retrieval tools that connect to Amazon Bedrock Knowledge Bases:
1. **Query Analysis**: Agent analyzes incoming query to determine type
2. **Tool Selection**: Routes to unstructured (document) or structured (SQL) assistant
3. **Retrieval**: Appropriate knowledge base retrieves relevant information
4. **Response Generation**: Agent synthesizes and presents the results

This approach enables handling both qualitative questions (business strategy, policies) and quantitative queries (financial metrics, data analysis) within a single conversational interface.

## Prerequisites and Setup

Before running this notebook, ensure you have completed:
1. **`0-prerequisites-structured-kb.ipynb`** - Sets up Redshift-based structured Amazon Bedrock Knowledge Base
2. **`1-prerequisites-unstructured-kb.ipynb`** - Sets up document-based unstructured Amazon Bedrock Knowledge Base


Let's start by importing the required libraries:


In [2]:
import os
import boto3
from strands import Agent, tool

In [3]:
# Set up AWS region and  Amazon Bedrock Agent Runtime client for knowledge base interactions
region = boto3.Session().region_name 
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=region)

### Load Knowledge Base Configuration

Load the knowledge base IDs from the prerequisite notebooks:


In [4]:

# Retrieve stored knowledge base IDs from prerequisite notebooks
%store -r unstructured_kb_id
%store -r structured_kb_id 
%store -r kb_region
%store -r structured_kb_region

# Use the stored values
UNSTRUCTURED_KB_ID = unstructured_kb_id  # From 1-prerequisites-unstructured-kb.ipynb
STRUCTURED_KB_ID = structured_kb_id      # From 0-prerequisites-structured-kb.ipynb


print("="*60)
print(f"Unstructured KB ID: {UNSTRUCTURED_KB_ID}")
print(f"Structured KB ID: {STRUCTURED_KB_ID}")
print(f"Unstructured KB Region: {kb_region}")
print(f"Structured KB Region: {structured_kb_region}")


## Custom Retrieval Tools

We will create two specialized tools using the Strands Agents `@tool` decorator. Each tool handles different types of queries by connecting to the appropriate knowledge base.

### Unstructured Data Assistant Tool

This tool handles document-based queries by retrieving information from the unstructured knowledge base (PDF documents, reports, policies):


In [5]:
@tool
def unstructured_data_assistant(query: str) -> str:
    """
    Handle document-based, narrative, and conceptual queries using the unstructured knowledge base.
    
    Args:
        query: A question about business strategies, policies, company information, 
               or requiring document comprehension and qualitative analysis
    
    Returns:
        Raw retrieve response from the unstructured knowledge base
    """
    try:
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=UNSTRUCTURED_KB_ID,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 10,
                }
            }
        )
        
        return retrieve_response
        
    except Exception as e:
        return f"Error in unstructured data assistant: {str(e)}"


### Structured Data Assistant Tool

This tool handles data analysis queries by retrieving information from the structured knowledge base (SQL/Redshift database):


In [6]:
@tool
def structured_data_assistant(query: str) -> str:
    """
    Handle data analysis, metrics, and quantitative queries using the structured knowledge base.
    
    Args:
        query: A question requiring calculations, aggregations, statistical analysis,
               or database operations on structured data
    
    Returns:
        Raw retrieve response from the structured knowledge base
    """
    try:
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=STRUCTURED_KB_ID,
            retrievalQuery={'text': query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 10,
                }
            }
        )
        
        return retrieve_response
        
    except Exception as e:
        return f"Error in structured data assistant: {str(e)}"


## Intelligent Agent with Query Routing

Now we'll create the orchestrator agent with our custom tools and system prompt that intelligently routes queries to the appropriate tool based on the query type and content.


In [7]:
# Create the orchestrator agent with both tools
orchestrator = Agent(
    system_prompt="""You are an intelligent assistant that routes queries to the appropriate knowledge base. Choose the appropriate tool based on the query type. 
    The tools return raw data that you should analyze and present in a clear, helpful format.""",
    tools=[
        unstructured_data_assistant,
        structured_data_assistant
    ]
)

Let's test our agent with different types of queries to observe how it routes them to the appropriate knowledge bases.

### Example 1: Unstructured Query

This query asks about qualitative, document-based information and should be routed to the unstructured knowledge base:


In [8]:
# EXAMPLE 1: Business Strategy Query (should use unstructured_data_assistant)
print("=== EXAMPLE 1: BUSINESS STRATEGY QUERY ===")
print("Query: What is Octank Financial's business strategy?")
print()

response = orchestrator("What is Octank Financial's business strategy?")
print(response)

=== EXAMPLE 1: BUSINESS STRATEGY QUERY ===
Query: What is Octank Financial's business strategy?

I'll help you find information about Octank Financial's business strategy by querying our knowledge base. This is a question about company information and business strategies, so I'll use the appropriate tool for that.
Tool #1: unstructured_data_assistant
# Octank Financial's Business Strategy

Based on the information retrieved from Octank Financial's documentation, here's a comprehensive overview of their business strategy:

## Core Business Focus
Octank Financial is a leading financial services company that provides a wide range of services including:
- Investment banking
- Wealth management
- Asset management
- Corporate finance
- Private equity

## Strategic Pillars

### 1. Client-Centric Approach
- Building long-term relationships with clients based on trust, transparency, and mutual respect
- Delivering exceptional service tailored to clients' unique needs and goals
- Working closely

### Example 2: Structured Query

This query asks for quantitative data analysis and should be routed to the structured knowledge base:


In [9]:
# EXAMPLE 2: Financial Data Query (should use structured_data_assistant)
print("=== EXAMPLE 2: FINANCIAL DATA QUERY ===")
print("Query: What is the total spending by all customers?")
print()

response = orchestrator("What is the total spending by all customers?")
print(response)


=== EXAMPLE 2: FINANCIAL DATA QUERY ===
Query: What is the total spending by all customers?

I'll help you find information about the total spending by all customers of Octank Financial. This question requires analysis of numerical data, so I'll use the structured data assistant to retrieve this information.
Tool #2: structured_data_assistant
# Total Customer Spending

Based on the query results from Octank Financial's database, the total spending by all customers is:

**$5,078,473.69** (Five million, seventy-eight thousand, four hundred seventy-three dollars and sixty-nine cents)

This figure represents the sum of all order totals in the company's transaction records.# Total Customer Spending

Based on the query results from Octank Financial's database, the total spending by all customers is:

**$5,078,473.69** (Five million, seventy-eight thousand, four hundred seventy-three dollars and sixty-nine cents)

This figure represents the sum of all order totals in the company's transaction

## Agent Thinking Inspection

One of the powerful features of Strands Agents is the ability to inspect the agent's internal reasoning process. Let's examine how the agent made its decisions:

In [12]:
# Inspect the complete conversation flow
def inspect_message_flow(messages):
    print("=== DETAILED MESSAGE FLOW ===")
    
    for i, message in enumerate(messages):
        print(f"\n--- Message {i+1} ---")
        print(f"Role: {message['role']}")
        
        for j, content in enumerate(message['content']):
            print(f"  Content {j+1}:")
            
            if 'text' in content:
                text = content['text']
                # Truncate long text for readability
                if len(text) > 200:
                    text = text[:200] + "..."
                print(f"    Text: {text}")
            
            elif 'toolUse' in content:
                tool_use = content['toolUse']
                print(f"    Tool Use: {tool_use['name']}")
                print(f"    Input: {tool_use['input']}")
                print(f"    ID: {tool_use['toolUseId']}")
            
            elif 'toolResult' in content:
                tool_result = content['toolResult']
                print(f"    Tool Result: {tool_result['status']}")
                print(f"    ID: {tool_result['toolUseId']}")
                # Don't print full content as it's very long
                print(f"    Content: [Raw KB Response - {len(str(tool_result['content']))} chars]")

# Run the inspection
inspect_message_flow(orchestrator.messages)


=== DETAILED MESSAGE FLOW ===

--- Message 1 ---
Role: user
  Content 1:
    Text: What is Octank Financial's business strategy?

--- Message 2 ---
Role: assistant
  Content 1:
    Text: I'll help you find information about Octank Financial's business strategy by querying our knowledge base. This is a question about company information and business strategies, so I'll use the appropri...
  Content 2:
    Tool Use: unstructured_data_assistant
    Input: {'query': "What is Octank Financial's business strategy?"}
    ID: tooluse_SsBe3pmZT5qzUkJEyqtNXw

--- Message 3 ---
Role: user
  Content 1:
    Tool Result: success
    ID: tooluse_SsBe3pmZT5qzUkJEyqtNXw
    Content: [Raw KB Response - 20776 chars]

--- Message 4 ---
Role: assistant
  Content 1:
    Text: # Octank Financial's Business Strategy

Based on the information retrieved from Octank Financial's documentation, here's a comprehensive overview of their business strategy:

## Core Business Focus
Oc...

--- Message 5 ---
Role: user
 

## Clean up the resources

When you have finished with this notebook, return to the previous notebook to delete the resources created and not incurr extra costs!